In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision.models import AlexNet_Weights

In [2]:
class AlexNetScratchPyTorch(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetScratchPyTorch, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # Cambiar tamaño del kernel
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 4096),  # Ajustar tamaño de entrada según dimensiones
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [3]:
# Cargar dataset CIFAR-10 con imágenes redimensionadas a 224x224
def load_cifar10(batch_size=32):
    transform = transforms.Compose([
        transforms.Resize(32),  # AlexNet espera imágenes de 224x224
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    return trainloader, testloader

In [4]:
# Función para evaluar el modelo
def evaluate_model_pytorch(model, testloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [5]:
# Inicializar modelo, criterio y optimizador
def train_alexnet_pytorch():
    trainloader, testloader = load_cifar10(batch_size=32)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AlexNetScratchPyTorch(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Entrenamiento del modelo
    model.train()
    epochs = 10
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Evaluación
    evaluate_model_pytorch(model, testloader)

In [6]:
# Entrenar el modelo desde cero
train_alexnet_pytorch() #5:33 min

100%|██████████| 170M/170M [00:12<00:00, 13.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1, Loss: 1.5860, Accuracy: 40.70%
Epoch 2, Loss: 1.2203, Accuracy: 55.83%
Epoch 3, Loss: 1.0603, Accuracy: 62.44%
Epoch 4, Loss: 0.9570, Accuracy: 66.40%
Epoch 5, Loss: 0.8821, Accuracy: 69.08%
Epoch 6, Loss: 0.8234, Accuracy: 71.22%
Epoch 7, Loss: 0.7809, Accuracy: 72.91%
Epoch 8, Loss: 0.7477, Accuracy: 74.12%
Epoch 9, Loss: 0.7109, Accuracy: 75.49%
Epoch 10, Loss: 0.6899, Accuracy: 76.12%
Test Accuracy: 70.93%


In [10]:
# Cargar dataset CIFAR-10 con imágenes redimensionadas a 224x224
def load_cifar10_2(batch_size=32):
    transform = transforms.Compose([
        transforms.Resize(224),  # AlexNet espera imágenes de 224x224
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    return trainloader, testloader

# Función para cargar el modelo preentrenado AlexNet y adaptarlo para CIFAR-10
def get_pretrained_alexnet_pytorch():
    # Carga el modelo AlexNet preentrenado utilizando los pesos IMAGENET1K_V1
    # Estos pesos corresponden a un modelo entrenado previamente en el conjunto de datos ImageNet (1000 clases)
    model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)

    # Modifica la última capa de la parte "classifier" del modelo
    # La arquitectura original de AlexNet tiene una última capa densa con 1000 salidas para las clases de ImageNet
    # Aquí, se reemplaza con una capa con 10 salidas (para las 10 clases de CIFAR-10)
    model.classifier[6] = nn.Linear(4096, 10)  # nn.Linear(entrada, salida)

    # Devuelve el modelo ajustado
    return model

# Entrenar el modelo preentrenado
def train_pretrained_alexnet_pytorch():
    trainloader, testloader = load_cifar10_2(batch_size=32)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_pretrained_alexnet_pytorch().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Entrenar el modelo
    model.train()
    epochs = 10
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # Evaluación
    evaluate_model_pytorch(model, testloader)


In [11]:
# Entrenar el modelo corregido
train_pretrained_alexnet_pytorch() # 18 min

Files already downloaded and verified
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 190MB/s]


Epoch 1, Loss: 2.3047, Accuracy: 10.00%
Epoch 2, Loss: 2.3034, Accuracy: 10.03%
Epoch 3, Loss: 2.3033, Accuracy: 9.91%
Epoch 4, Loss: 2.3033, Accuracy: 9.85%
Epoch 5, Loss: 2.3032, Accuracy: 10.00%
Epoch 6, Loss: 2.3033, Accuracy: 10.02%
Epoch 7, Loss: 2.3031, Accuracy: 10.25%
Epoch 8, Loss: 2.3032, Accuracy: 9.93%
Epoch 9, Loss: 2.3032, Accuracy: 9.89%
Epoch 10, Loss: 2.3031, Accuracy: 9.84%
Test Accuracy: 10.00%
